In [1]:
from threading import Thread
import json
from stmpy import Machine, Driver
import paho.mqtt.client as mqtt
from utils import BROKER, PORT, TOPIC


class MQTT_Client_1:
    def __init__(self, stm):
        self.state_machine = stm
        self.count = 0
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        self.stm_driver: Driver = None

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        try:
            message = json.loads(msg.payload.decode('utf-8'))["msg"]
        except json.decoder.JSONDecodeError:
            return
                    
        print("on_message(): topic: {}".format(msg.topic))

        if message == "session_created":
            self.stm_driver.send("session_created", "teacher")
        
    def start(self, broker, port):
        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)
        self.client.subscribe(TOPIC)

        try:
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

In [2]:
from stmpy import Driver, Machine
from threading import Thread
import paho.mqtt.client as mqtt
import ipywidgets as widgets
from IPython.display import display
import random

class Teacher:
    def __init__(self):
        self.button_create = widgets.Button(description="Create Session")
        self.button_create.on_click(self.create_session)
        self.button_join = widgets.Button(description="Join Session")
        self.button_join.on_click(self.join_session)

        self.mqtt_client: mqtt.Client = None
        self.stm: Machine = None
        
        # text field
        self.ta_code = widgets.Text(value='', placeholder='', description='TA code:', disabled=False)
        display(self.button_create, self.ta_code, self.button_join)

        display()

    def create_session(self, b):
        """
        Create session. Send message to server and receive codes indicating the session
        has been created.
        """
        self.stm.send("start_session")
        pass
    
    def join_session(self, b):
        """Join an already existing session using a TA code."""

        pass
    
    def in_wait(self):
        """Called upon entering wait-state."""
        print("In wait-state")
        return
    
    def in_lab_session_active(self):
        """Called upon entering lab_sesssion_active-state."""
        self.button_help = widgets.Button(description="Help")
        self.button_help.on_click(self.help)
        self.button_end = widgets.Button(description="End lab session")
        self.button_end.on_click(self.end_session)
        display(self.button_help, self.button_end)

        print("In lab_sesssion_active-state")

    def in_help(self):
        """Called upon entering help-state."""  
        self.button_help = widgets.Button(description="Finished Helping")
        self.button_help.on_click(self.finish_help)
        display(self.button_help)

    def help(self, b):
        """Called when Help-button is pressed in lab_session_active"""
        # Inform server that group is receiving help.
        print(b)
        self.mqtt_client.publish(TOPIC, "helping")
        self.stm.send("help_group")

    def finish_help(self, b):
        """Called when Finished Help-button is pressed in help"""
        # Inform server that group has been helped successfully.
        self.mqtt_client.publish(TOPIC, "finished helping")
        self.stm.send("finish_help")

    def end_session(self, b):
        """Called when End Session-button is pressed in lab_session_active"""
        # Commit Sudoku
        self.stm.send("end_lab")
        

# initial transition
t0 = {'source': 'initial',
      'target': 'idle'}

# transitions
t1 = {'trigger':'start_session', 
      'source':'idle', 
      'target':'wait'}
t2 = {'trigger':'session_created', 
      'source':'wait', 
      'target':'lab_session_active'}

t3 = {'trigger':'help_group', 
      'source':'lab_session_active', 
      'target':'help'}
t4 = {'trigger':'finish_help', 
      'source':'help', 
      'target':'lab_session_active'}

t5 = {'trigger':'task_done', 
      'source':'lab_session_active', 
      'target':'lab_session_active'}
t6 = {'trigger':'update_queue', 
      'source':'lab_session_active', 
      'target':'lab_session_active'}

t7 = {'trigger':'end_lab', 
      'source':'lab_session_active', 
      'target':'exit'}
 
# the states:
idle = {'name': 'idle'}

wait = {'name': 'wait',
        'entry': 'in_wait'}

lab_session_active = {'name': 'lab_session_active',
        'entry': 'in_lab_session_active'}

help = {'name': 'help',
        'entry': 'in_help'}



In [3]:
teacher = Teacher()
state_machine = Machine(transitions=[t0, t1, t2, t3, t4, t5, t6, t7], states=[idle, wait, lab_session_active, help], obj=teacher, name="teacher")
teacher.stm = state_machine

driver = Driver()
driver.add_machine(state_machine)

myclient = MQTT_Client_1(teacher)
teacher.mqtt_client = myclient.client
myclient.stm_driver = driver

driver.start()
myclient.start(BROKER, PORT)

Button(description='Create Session', style=ButtonStyle())

Text(value='', description='TA code:', placeholder='')

Button(description='Join Session', style=ButtonStyle())

Connecting to mqtt20.iik.ntnu.no:1883


on_connect(): Connection Accepted.
on_message(): topic: team02
on_message(): topic: team02
on_message(): topic: team02
In wait-state


Machine teacher is in state wait and received event start_session, but no transition with this event is declared!


on_message(): topic: team02
on_message(): topic: team02
on_message(): topic: team02
on_connect(): Connection Accepted.
on_connect(): Connection Accepted.


Machine teacher is in state wait and received event start_session, but no transition with this event is declared!
Machine teacher is in state wait and received event start_session, but no transition with this event is declared!
Machine teacher is in state wait and received event start_session, but no transition with this event is declared!
Machine teacher is in state wait and received event start_session, but no transition with this event is declared!
Machine teacher is in state wait and received event start_session, but no transition with this event is declared!
Machine teacher is in state wait and received event start_session, but no transition with this event is declared!
Machine teacher is in state wait and received event start_session, but no transition with this event is declared!
Machine teacher is in state wait and received event start_session, but no transition with this event is declared!
Machine teacher is in state wait and received event start_session, but no transition wit